# 06. Python “黑箱”：输入与输出

## 输入输出基础

最简单直接的输入来自键盘操作

In [2]:
name = input('your name:')
gender = input('you are a boy?(y/n)')

your name:Jianhua
you are a boy?(y/n)y


In [5]:
welcome_str = 'Welcome to the matrix {prefix} {name}.'
welcome_dic = {
    'prefix': 'Mr.' if gender == 'y' else 'Mrs',
    'name': name 
}

print('authorizing...')
print(welcome_str.format(**welcome_dic))

authorizing...
Welcome to the matrix Mr. Jianhua.


input函数的参数即为提示语，**输入的类型永远是字符串型（str）**。注意，初学者在这里很容易犯错，下面的例子我会讲到。print() 函数则接受字符串、数字、字典、列表甚至一些自定义类的输出。

In [6]:
 a = input()

1


In [7]:
b = input()

2


In [8]:
print('a + b = {}'.format(a + b))

a + b = 12


In [9]:
print('type of a is {}, type of b is {}'.format(type(a), type(b)))

type of a is <class 'str'>, type of b is <class 'str'>


In [10]:
print('a + b = {}'.format(int(a) + int(b)))

a + b = 3


注意： 虽然输入输出和类型处理事情简单，但我们一定要慎之又慎。毕竟相当比例的安全漏洞，都来自随意的 I/O 处理。
1. 在生产环境中使用强制转换时，请记得加上 try except（即错误和异常处理，专栏后面文章会讲到）。
2. Python 对 int 类型没有最大限制（相比之下， C++ 的 int 最大为 2147483647，超过这个数字会产生溢出），但是对 float 类型依然有精度限制。这些特点，在生产环境中要时刻提防，避免因为对边界条件判断不清而造成 bug 甚至 0day（危重安全漏洞）。

## 文件输入输出

生产级别的 Python 代码，大部分 I/O 则来自于文件、网络、其他进程的消息等等

我们来做一个简单的 NLP（自然语言处理）任务

首先，我们要清楚 NLP 任务的基本步骤，也就是下面的四步：
1. 读取文件；
2. 去除所有标点符号和换行符，并把所有大写变成小写；
3. 合并相同的词，统计每个词出现的频率，并按照词频从大到小排序；
4. 将结果按行输出到文件 out.txt。

In [12]:
import re 

def parse(text):
    # 使用正则表达式去除标点符号和换行符
    text = re.sub(r'[^\w ]', ' ', text)
    
    # 转为小写
    text = text.lower()
    
    # 生成所有单词的列表
    word_list = text.split(' ')
    
    # 去除空白单词
    word_list = filter(None, word_list)
    
    # 生成单词和词频的字典
    word_cnt = {}
    
    for word in word_list:
        if word not in word_cnt:
            word_cnt[word] = 0 
        word_cnt[word] += 1
    
    # 按照词频排序
    sorted_word_cnt = sorted(word_cnt.items(), key=lambda kv: kv[1], reverse=True)
    
    return sorted_word_cnt
    

with open('in.txt', 'r') as fin:
    text = fin.read()
    
word_and_freq = parse(text)

with open('out.txt', 'w') as fout:
    for word, freq in word_and_freq:
        fout.write('{} {}\n'.format(word, freq))

计算机中文件访问的基础知识

1. 先要用 open() 函数拿到文件的指针(fin)。其中，
- 第一个参数指定文件位置（相对位置或者绝对位置）；
- 第二个参数，如果是 'r'表示读取，如果是'w' 则表示写入，当然也可以用 'rw' ，表示读写都要。a 则是一个不太常用（但也很有用）的参数，表示追加（append），这样打开的文件，如果需要写入，会从原始文件的最末尾开始写入。

注意：在工作中，代码权限管理非常重要。如果你只需要读取文件，就不要请求写入权限。这样在某种程度上可以降低 bug 对整个系统带来的风险。

2. 在拿到指针后，我们可以通过 read() 函数，来读取文件的全部内容。代码 text = fin.read() ，即表示把文件所有内容读取到内存中，并赋值给变量 text。这么做自然也是有利有弊：
     - 优点是方便，接下来我们可以很方便地调用 parse 函数进行分析；
     - 缺点是如果文件过大，一次性读取可能造成内存崩溃。这时，我们可以给 read 指定参数 size ，用来表示读取的最大长度。还可以通过 readline() 函数，每次读取一行，这种做法常用于数据挖掘（Data Mining）中的数据清洗
     
3. open() 函数对应于 close() 函数，打开了文件，在完成读取任务后，就应该立刻关掉它。而如果你使用了 with 语句，就不需要显式调用 close()

4. 所有 I/O 都应该进行错误处理。因为 I/O 操作可能会有各种各样的情况出现，而一个健壮（robust）的程序，需要能应对各种情况的发生，而不应该崩溃

## JSON序列化与实战

JSON，可以把它简单地理解为两种黑箱：
- 第一种，输入这些杂七杂八的信息，比如 Python 字典，输出一个字符串；
- 第二种，输入这个字符串，可以输出包含原始信息的 Python 字典。

In [13]:
import json

params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}

In [14]:
params_str = json.dumps(params)

print('after json serialization')
print('type of params_str = {}, params_str = {}'.format(type(params_str), params))

after json serialization
type of params_str = <class 'str'>, params_str = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


In [15]:
original_params = json.loads(params_str)

print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


In [16]:
params_str

'{"symbol": "123456", "type": "limit", "price": 123.4, "amount": 23}'

In [17]:
original_params

{'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}

1. json.dumps() 这个函数，接受 Python 的基本数据类型，然后将其序列化为 string；
2. 而 json.loads() 这个函数，接受一个合法字符串，然后将其反序列化为 Python 的基本数据类型。

如果我要输出字符串到文件，或者从文件中读取 JSON 字符串，又该怎么办呢？

In [18]:
import json

params = { 'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}

with open('params.json', 'w') as fout: 
    params_str = json.dump(params, fout)
    
with open('params.json', 'r') as fin: 
    original_params = json.load(fin)
    
print('after json deserialization')
print('type of original_params = {}, original_params = {}'.format(type(original_params), original_params))

after json deserialization
type of original_params = <class 'dict'>, original_params = {'symbol': '123456', 'type': 'limit', 'price': 123.4, 'amount': 23}


当开发一个第三方应用程序时，你可以通过 JSON 将用户的个人配置输出到文件，方便下次程序启动时自动读取。这也是现在普遍运用的成熟做法。

在 Google，有类似的工具叫做 Protocol Buffer. 相比于 JSON，它的优点是生成优化后的二进制文件，因此性能更好。但与此同时，生成的二进制序列，是不能直接阅读的。它在 TensorFlow 等很多对性能有要求的系统中都有广泛的应用。

## 总结

我们主要学习了 Python 的普通 I/O 和文件 I/O，同时了解了 JSON 序列化的基本知识，并通过具体的例子进一步掌握。再次强调一下需要注意的几点：

1. I/O 操作需谨慎，一定要进行充分的错误处理，并细心编码，防止出现编码漏洞；
2. 编码时，对内存占用和磁盘占用要有充分的估计，这样在出错时可以更容易找到原因；
3. JSON 序列化是很方便的工具，要结合实战多多练习；
4. 代码尽量简洁、清晰，哪怕是初学阶段，也要有一颗当元帅的心。

## 思考题

最后，我给你留了两道思考题。

1. 第一问：你能否把 NLP 例子中的 word count 实现一遍？不过这次，in.txt 可能非常非常大（意味着你不能一次读取到内存中），而 output.txt 不会很大（意味着重复的单词数量很多）。提示：你可能需要每次读取一定长度的字符串，进行处理，然后再读取下一次的。但是如果单纯按照长度划分，你可能会把一个单词隔断开，所以需要细心处理这种边界情况。

2. 第二问：你应该使用过类似百度网盘、Dropbox 等网盘，但是它们可能空间有限（比如 5GB）。如果有一天，你计划把家里的 100GB 数据传送到公司，可惜你没带 U 盘，于是你想了一个主意：每次从家里向 Dropbox 网盘写入不超过 5GB 的数据，而公司电脑一旦侦测到新数据，就立即拷贝到本地，然后删除网盘上的数据。等家里电脑侦测到本次数据全部传入公司电脑后，再进行下一次写入，直到所有数据都传输过去。

In [20]:
# ’不瘦到140不改名‘回复
from collections import defaultdict
import re

f = open("in.txt", mode="r", encoding="utf-8")
d = defaultdict(int)

for line in f:
    for word in filter(lambda x: x, re.split(r"\s", line)):
        d[word] += 1


print(d)

defaultdict(<class 'int'>, {'I': 5, 'have': 5, 'a': 8, 'dream': 5, 'that': 5, 'my': 1, 'four': 1, 'little': 3, 'children': 1, 'will': 11, 'one': 5, 'day': 5, 'live': 1, 'in': 4, 'nation': 2, 'where': 1, 'they': 1, 'not': 1, 'be': 13, 'judged': 1, 'by': 2, 'the': 10, 'color': 1, 'of': 10, 'their': 2, 'skin': 1, 'but': 1, 'content': 1, 'character.': 1, 'today.': 2, 'down': 1, 'Alabama,': 1, 'with': 2, 'its': 1, 'vicious': 1, 'racists,': 1, '.': 8, 'right': 1, 'there': 1, 'Alabama': 1, 'black': 3, 'boys': 2, 'and': 14, 'girls': 2, 'able': 6, 'to': 11, 'join': 2, 'hands': 2, 'white': 3, 'as': 1, 'sisters': 1, 'brothers.': 1, 'every': 6, 'valley': 1, 'shall': 4, 'exalted,': 1, 'hill': 1, 'mountain': 2, 'made': 3, 'low,': 1, 'rough': 1, 'places': 2, 'plain,': 1, 'crooked': 1, 'straight,': 1, 'glory': 1, 'Lord': 1, 'revealed,': 1, 'all': 2, 'flesh': 1, 'see': 1, 'it': 2, 'together.': 1, 'This': 1, 'is': 1, 'our': 2, 'hope.': 2, 'With': 3, 'this': 4, 'faith': 3, 'we': 8, 'hew': 1, 'out': 1, 'd

In [23]:
# ’Geek_59f23e‘回复
# 第一题：
# 1、使用defaultdict初始化计数器更方便更快，不用再多做一步in判断，parse函数只需返回filter对象。

# 2、读取大文件时使用for循环遍历迭代器，不占用内存空间，生成一行处理一行，
# 就此例来说每一行行尾都是\n没有跨行单词，故此方法不用考虑边界问题，因文件中有多行\n，读取时做一步判断跳过避免再调用parse函数。

import re 

def parse(text):
    # 使用正则表达式去除标点符号和换行符
    text = re.sub(r'[^\w ]', ' ', text)
    
    # 转为小写
    text = text.lower()
    
    # 生成所有单词的列表
    word_list = text.split(' ')
    
    # 去除空白单词
    word_list = filter(None, word_list)
    
    return word_list

word_cnt = defaultdict(lambda: 0)

with open('in.txt', 'r') as fin:
    for line in fin:
        if line != '\n':
            for word in parse(line):
                word_cnt[word] += 1
                
print(word_cnt)

defaultdict(<function <lambda> at 0x1166eeea0>, {'i': 5, 'have': 5, 'a': 8, 'dream': 5, 'that': 5, 'my': 1, 'four': 1, 'little': 3, 'children': 2, 'will': 11, 'one': 5, 'day': 6, 'live': 1, 'in': 4, 'nation': 2, 'where': 1, 'they': 1, 'not': 1, 'be': 13, 'judged': 1, 'by': 2, 'the': 10, 'color': 1, 'of': 10, 'their': 2, 'skin': 1, 'but': 1, 'content': 1, 'character': 1, 'today': 2, 'down': 1, 'alabama': 2, 'with': 5, 'its': 1, 'vicious': 1, 'racists': 1, 'right': 1, 'there': 1, 'black': 3, 'boys': 2, 'and': 15, 'girls': 2, 'able': 6, 'to': 11, 'join': 2, 'hands': 2, 'white': 3, 'as': 1, 'sisters': 1, 'brothers': 1, 'every': 6, 'valley': 1, 'shall': 4, 'exalted': 1, 'hill': 1, 'mountain': 2, 'made': 3, 'low': 1, 'rough': 1, 'places': 2, 'plain': 1, 'crooked': 1, 'straight': 1, 'glory': 1, 'lord': 1, 'revealed': 1, 'all': 2, 'flesh': 1, 'see': 1, 'it': 2, 'together': 6, 'this': 5, 'is': 1, 'our': 2, 'hope': 2, 'faith': 3, 'we': 8, 'hew': 1, 'out': 1, 'despair': 1, 'stone': 1, 'transform'